#### Test Pipeline
This is a Julia Notebook to check the correctness of the C++ code. 

The notebook proceeds as follows: 
- Generate a synthetic instance
- Run the original implementation of the algorithm (pure Julia)
- Run the C++ implementation of the algorithm

#### Step 1: Data Generation

Generates a ground truth covariance matrix S of the form $I + \beta x_1 x_1^\top + \beta x_2 x_2^\top$, where 
- $x_1, x_2$ are $k$-sparse, with non-overlapping support 
- $\beta$ controls the signal-to-noise ratio

Then, samples $n$ multivariate normal observation from $\mathcal{N}(0,S)$ and constructs the empirical covariance matrix $\Sigma$

In [39]:
using Random, LinearAlgebra
p = 10 #Dimension
r = 2 #Number of sparse PCs
k = 4 #Sparsity of each PC
β = 1 #Signal strength


x1 = zeros(p); x1[1:k] = sign.(rand(k) .- .5)
x2 = zeros(p); x2[(k+1):(k+k)] = sign.(rand(k) .- .5)

# x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap)] = sign.(rand(k_overlap) .- .5)
# x2[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)] = -x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)]
# x2[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)] = x1[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)]

@assert sum(abs.(x1) .> 0) == k
@assert sum(abs.(x2) .> 0) == k
@assert abs(dot(x1,x2)) ≤ 1e-10

shufflecoords = randperm(p)
x1 = x1[shufflecoords]; x2=x2[shufflecoords] 

x1 /= norm(x1); x2 /= norm(x2) 

S = β*x1*x1'+β*x2*x2'+ Matrix(1.0*I, p, p)
S = (S + S')/2

10×10 Matrix{Float64}:
  1.25   0.0    0.0   0.0   0.25  -0.25  -0.25   0.0    0.0   0.0
  0.0    1.25  -0.25  0.0   0.0    0.0    0.0    0.25  -0.25  0.0
  0.0   -0.25   1.25  0.0   0.0    0.0    0.0   -0.25   0.25  0.0
  0.0    0.0    0.0   1.0   0.0    0.0    0.0    0.0    0.0   0.0
  0.25   0.0    0.0   0.0   1.25  -0.25  -0.25   0.0    0.0   0.0
 -0.25   0.0    0.0   0.0  -0.25   1.25   0.25   0.0    0.0   0.0
 -0.25   0.0    0.0   0.0  -0.25   0.25   1.25   0.0    0.0   0.0
  0.0    0.25  -0.25  0.0   0.0    0.0    0.0    1.25  -0.25  0.0
  0.0   -0.25   0.25  0.0   0.0    0.0    0.0   -0.25   1.25  0.0
  0.0    0.0    0.0   0.0   0.0    0.0    0.0    0.0    0.0   1.0

In [34]:
n = 1000 #Sample size

using Distributions
d = MvNormal(zeros(p), S)
X = rand(d, n) #p by N matrix of observations

Σ = cov(X') #Sample covariance matrix

10×10 Matrix{Float64}:
  1.4215     -0.177758     0.171349    …  -0.200763     0.193598
 -0.177758    1.14456      0.00927575      0.196576    -0.0262446
  0.171349    0.00927575   1.11214         0.0176324    0.169316
  0.035725    0.0191874    0.0690245       0.0143123    0.0217149
  0.0367641  -0.0151243   -0.0411098      -0.0148459    0.00556542
  0.228111   -0.211712    -0.0199956   …  -0.205748     0.0457019
 -0.203184   -0.0150608   -0.197376       -0.0274892   -0.238753
 -0.0120782  -0.256536    -0.21512        -0.223914    -0.210773
 -0.200763    0.196576     0.0176324       1.18265      0.00768102
  0.193598   -0.0262446    0.169316        0.00768102   1.19648

In [45]:
[x1 x2]

10×2 Matrix{Float64}:
  0.5   0.0
  0.0   0.5
  0.0  -0.5
  0.0   0.0
  0.5   0.0
 -0.5   0.0
 -0.5   0.0
  0.0   0.5
  0.0  -0.5
  0.0   0.0

#### Step 2: Julia benchmark

Applies the Julia code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [48]:
include("../julia/algorithm2.jl")

ofv_best, violation_best, runtime, x_best = findmultPCs_deflation(S, r, [k,k]; numIters = 20, verbose = true, violation_tolerance = 1e-4 )

x_best

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern: [4, 4]

  Iteration |      Objective value |   Orthogonality Violation |       Time 
          1 |                0.333 |                  2.00e+00 |      0.298 


          2 |                0.333 |                  1.00e-07 |      0.322 


10×2 Matrix{Float64}:
  0.0   0.5
 -0.5   0.0
  0.5   0.0
  0.0   0.0
  0.0   0.5
  0.0  -0.5
  0.0  -0.5
 -0.5   0.0
  0.5   0.0
  0.0   0.0

In [50]:
ofv_best, violation_best, runtime, x_best = findmultPCs_deflation(Σ, r, [k,k]; numIters = 20, verbose = true, violation_tolerance = 1e-4 )

x_best

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern: [4, 4]

  Iteration |      Objective value |   Orthogonality Violation |       Time 
          1 |                0.318 |                  2.00e+00 |      0.040 


          2 |                0.318 |                  2.00e+00 |      0.064 
          3 |                0.316 |                  1.24e+00 |      0.098 
          4 |                0.315 |                  1.22e+00 |      0.124 
          5 |                0.313 |                  6.85e-01 |      0.157 


          6 |                0.313 |                  6.67e-01 |      0.192 
          7 |                0.311 |                  1.00e-07 |      0.220 
          8 |                0.311 |                  1.00e-07 |      0.256 
          9 |                0.311 |                  1.00e-07 |      0.286 


         10 |                0.311 |                  1.00e-07 |      0.323 
         11 |                0.311 |                  1.00e-07 |      0.362 


         12 |                0.311 |                  1.00e-07 |      0.390 
         13 |                0.311 |                  1.00e-07 |      0.428 
         14 |                0.311 |                  1.00e-07 |      0.457 


10×2 Matrix{Float64}:
  0.0        0.598201
  0.0       -0.430507
 -0.40799    0.0
  0.0        0.0
  0.0        0.0
  0.0        0.513878
  0.483613   0.0
  0.628464   0.0
  0.0       -0.439031
 -0.452433   0.0

#### Step 3: C++ implementation

Applies the C++ code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

See https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/ for documentation on calling C code in Julia

In [58]:
ofv_best = Ref{Cdouble}(0)
violation_best = Ref{Cdouble}(0)
runtime = Ref{Cdouble}(0)
x_best = zeros(Cdouble, p * r)
ptr_best = pointer(x_best)
numIters = 20 
verbose = 1
violation_tolerance = 1e-4 

@ccall "../c++/algorithm2.so".findmultPCs_deflation(
   ofv_best :: Ptr{Cdouble},
   violation_best :: Ptr{Cdouble},
   runtime :: Ptr{Cdouble},
   pointer(x_best) :: Ptr{Cdouble},
   n :: Cint,
   r :: Cint,
   pointer(reinterpret(Cdouble, reshape(S, p * p))) :: Ptr{Cdouble},
   pointer(reinterpret(Cint, [k, k])) :: Ptr{Cint},
   numIters :: Cint,
   verbose :: Cint,
   violation_tolerance :: Cdouble
)::Cvoid;

ofv_best = ofv_best[]::Float64
violation_best = violation_best[]::Float64
runtime = runtime[]::Float64
println(x_best)
x_best = reshape(reinterpret(Float64, x_best), n, r)